<a href="https://colab.research.google.com/github/prashantmalan/Chatbot/blob/main/XmltoDf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Assuming the XML file is named 'data.xml'
tree = ET.parse('/content/drive/My Drive/Colab Notebooks/anomaly/ird-ex01-vanilla-swap.xml')
root = tree.getroot()

def get_data_for_xpaths(element, current_path="", data=None, counter=None):
  """
  Recursively extracts data for each XPath, handling duplicates with a global counter.

  Args:
    element: The current XML element.
    current_path: The XPath expression leading to the current element.
    data: A dictionary to store the extracted data.
    counter: A dictionary to keep track of duplicate paths.

  Returns:
    A dictionary where keys are unique XPaths and values are data.
  """
  if data is None:
    data = {}
  if counter is None:
    counter = {}
  if not current_path:
    current_path = element.tag

  attrib_str = ""
  if element.attrib:
    attrib_str = " ".join([f"@{k}='{v}'" for k, v in element.attrib.items()])
  path_with_attrib = f"{current_path} [{attrib_str}]"

  if path_with_attrib not in counter:
    counter[path_with_attrib] = 0
  else:
    counter[path_with_attrib] += 1
  unique_path = f"{path_with_attrib}{counter[path_with_attrib]}" if counter[path_with_attrib] > 0 else path_with_attrib

  data[unique_path] = element.text

  for child in element:
    child_path = f"{unique_path}/{child.tag}"
    get_data_for_xpaths(child, child_path, data, counter)  # Pass the same counter down
  return data

# Get data for XPaths
data = get_data_for_xpaths(root)

# Create DataFrame
df = pd.DataFrame(data, index=[0])
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd'],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}header [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}header []/{http://www.fpml.org/FpML-5/transparency}messageId [@messageIdScheme='http://abc.com/message-id'],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}header []/{http://www.fpml.org/FpML-5/transparency}sentBy [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}header []/{http://www.fpml.org/FpML-5/transparency}sendTo [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}header []/{http://www.fpml.org/FpML-5/transparency}creationTimestamp [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}isCorrection [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}correlationId [@correlationIdScheme='http://abc.com/correlation-id'],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}sequenceNumber [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}originatingEvent [],...,{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}trade []/{http://www.fpml.org/FpML-5/transparency}swap []/{http://www.fpml.org/FpML-5/transparency}swapStream []1/{http://www.fpml.org/FpML-5/transparency}paymentDates []/{http://www.fpml.org/FpML-5/transparency}paymentFrequency []/{http://www.fpml.org/FpML-5/transparency}period [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}trade []/{http://www.fpml.org/FpML-5/transparency}swap []/{http://www.fpml.org/FpML-5/transparency}swapStream []1/{http://www.fpml.org/FpML-5/transparency}calculationPeriodAmount [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLoc

In [15]:
import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Assuming the XML file is named 'data.xml'
tree = ET.parse('/content/drive/My Drive/Colab Notebooks/anomaly/ird-ex01-vanilla-swap.xml')
root = tree.getroot()

def get_xpath_and_last_part_of_tag(element, current_path=""):
  """
  Recursively generates XPath expressions and the last part of tag names after '}'.

  Args:
    element: The current XML element.
    current_path: The XPath expression leading to the current element.

  Returns:
    A list of tuples, each containing (XPath, last part of tag name).
  """
  xpaths_and_tags = []
  if current_path:
    if '}' in element.tag:
      xpaths_and_tags.append((current_path, element.tag.split('}')[-1]))
    else:
      xpaths_and_tags.append((current_path, element.tag))

  for child in element:
    child_path = f"{current_path}/{child.tag}"
    xpaths_and_tags.extend(get_xpath_and_last_part_of_tag(child, child_path))
  return xpaths_and_tags

# Get XPaths and tag names
xpaths_and_tags = get_xpath_and_last_part_of_tag(root)

# Create DataFrame
df = pd.DataFrame(xpaths_and_tags, columns=['XPath', 'Field Name'])
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,XPath,Field Name
0,/{http://www.fpml.org/FpML-5/transparency}header,header
1,/{http://www.fpml.org/FpML-5/transparency}head...,messageId
2,/{http://www.fpml.org/FpML-5/transparency}head...,sentBy
3,/{http://www.fpml.org/FpML-5/transparency}head...,sendTo
4,/{http://www.fpml.org/FpML-5/transparency}head...,creationTimestamp
...,...,...
67,/{http://www.fpml.org/FpML-5/transparency}trad...,initialValue
68,/{http://www.fpml.org/FpML-5/transparency}trad...,currency
69,/{http://www.fpml.org/FpML-5/transparency}trad...,fixedRateSchedule
70,/{http://www.fpml.org/FpML-5/transparency}trad...,initialValue


In [30]:
import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Assuming the XML file is named 'data.xml'
tree = ET.parse('/content/drive/My Drive/Colab Notebooks/anomaly/ird-ex01-vanilla-swap.xml')
root = tree.getroot()

def get_data_for_xpaths(element, current_path="", data=None, counter=None):
  """
  Recursively extracts data for each XPath, handling duplicates dynamically.

  Args:
    element: The current XML element.
    current_path: The XPath expression leading to the current element.
    data: A dictionary to store the extracted data.
    counter: A dictionary to keep track of duplicate paths.

  Returns:
    A dictionary where keys are unique XPaths and values are data.
  """
  if data is None:
    data = {}
  if counter is None:
    counter = {}
  if not current_path:
    current_path = element.tag

  attrib_str = ""
  if element.attrib:
    attrib_str = " ".join([f"@{k}='{v}'" for k, v in element.attrib.items()])
  path_with_attrib = f"{current_path} [{attrib_str}]"

  if path_with_attrib not in counter:
    counter[path_with_attrib] = 0
  else:
    counter[path_with_attrib] += 1
  unique_path = f"{path_with_attrib}{counter[path_with_attrib]}" if counter[path_with_attrib] > 0 else path_with_attrib

  data[unique_path] = element.text

  for child in element:
    child_path = f"{unique_path}/{child.tag}"
    get_data_for_xpaths(child, child_path, data, counter.copy())
  return data

# Get data for XPaths
data = get_data_for_xpaths(root)

# Create DataFrame
df = pd.DataFrame(data, index=[0])
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd'],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}header [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}header []/{http://www.fpml.org/FpML-5/transparency}messageId [@messageIdScheme='http://abc.com/message-id'],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}header []/{http://www.fpml.org/FpML-5/transparency}sentBy [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}header []/{http://www.fpml.org/FpML-5/transparency}sendTo [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}header []/{http://www.fpml.org/FpML-5/transparency}creationTimestamp [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}isCorrection [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}correlationId [@correlationIdScheme='http://abc.com/correlation-id'],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}sequenceNumber [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}originatingEvent [],...,{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}trade []/{http://www.fpml.org/FpML-5/transparency}swap []/{http://www.fpml.org/FpML-5/transparency}swapStream []/{http://www.fpml.org/FpML-5/transparency}calculationPeriodAmount []/{http://www.fpml.org/FpML-5/transparency}calculation []/{http://www.fpml.org/FpML-5/transparency}floatingRateCalculation []/{http://www.fpml.org/FpML-5/transparency}indexTenor []/{http://www.fpml.org/FpML-5/transparency}periodMultiplier [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}trade []/{http://www.fpml.org/FpML-5/transparency}swap []/{http://www.fpml.org/FpML-5/transparency}swapStream []/{http://www.fpml.org/FpML-5/transparency}calculationPeriodAmount []